# Description
I got all the data and timestamp values\
To do this i checked that every element had unique `top` values in the style attribute so i used this as a criteria\
so as to not visit the element twice.\
If visited but not got the timestamp value then the element can be visited again.\
I scrolled the page by small values rather than large values so as to not skip some values\

In [1]:
import pandas as pd
import requests
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
import numpy as np
import time
import multiprocessing


pd.set_option('display.max_rows', None)

In [2]:
url = "https://opensea.io/nickname808/activity?search[eventTypes][0]=AUCTION_CREATED&search[eventTypes][1]=OFFER_ENTERED"

In [3]:
#get the top value of elements
def get_top(style):
    idx = style.index("top")
    top = ""
    for px in style[idx+5: ]:
        if px == ";":
            break
        top += px
    return top

In [ ]:
%%time
visited = []
err = []
data_str = []
driver = webdriver.Chrome() #generate a driver
driver.implicitly_wait(10)
driver.get(url)               #going to the url
driver.maximize_window()     #maximizes the window
time.sleep(3)                #waits 2 seconds after loading
screen_height = driver.execute_script("return window.screen.height;") #get current screen height
i = 0
sol = ""
#This loop will run until we reach the end of the infinite scroll
action = ActionChains(driver)
while True:
    #scroll the page to the next screen height one by one
    driver.execute_script("window.scrollTo(0, {screen_height}+{i});".format(screen_height=screen_height, i=i))
    i += 300
    #get all the text values
    divs = driver.find_elements(By.TAG_NAME, "div")
    for div in divs:
        try:
            s = ""
            if div.get_attribute("role") == "listitem":
                top = get_top(div.get_attribute("style"))
                if top not in visited:
                    visited.append(top)
                    if div.text != '':
                        s += div.text 
                        s += "\n"
                    spans = div.find_elements(By.TAG_NAME, "span")
                    for span in spans:
                        if span.get_attribute("aria-expanded") == "false":
                            action.move_to_element(span).perform()
                            try:
                                stamp = driver.find_element(By.CLASS_NAME, "tippy-content").text
                                s += stamp
                            except:
                                s += "NULL"
                                err.append(top)
                    if "NULL" not in s:
                        if s not in data_str:
                            data_str.append(s)
                    else:
                        visited.remove(top)
        except:
            continue
    #get current scrollHeight
    scroll_height = driver.execute_script("return document.body.scrollHeight;")
    #break the loop if the screen height exceeds the scroll height
    if (screen_height) + i > scroll_height:
        break
driver.close()

In [8]:
len(data_str)

185

In [9]:
data_str

['Offer\nLunar Calendar\nSunflower Land Collectibles\n3.900 USDC\n$3.90\n--\n4\nnickname808\n---\n1d ago\nApril 4, 2023 at 4:52 PM',
 'Offer\nLunar Calendar\nSunflower Land Collectibles\n3.700 USDC\n$3.70\n--\n2\nnickname808\n---\n1d ago\nApril 4, 2023 at 3:02 PM',
 'Offer\nCanceled\nLunar Calendar\nSunflower Land Collectibles\n3.820 USDC\n$3.82\n--\n5\nnickname808\n---\n2d ago\nApril 3, 2023 at 9:10 PM',
 'Offer\nCanceled\nLunar Calendar\nSunflower Land Collectibles\n3.800 USDC\n$3.80\n--\n5\nnickname808\n---\n2d ago\nApril 3, 2023 at 3:55 PM',
 'Offer\nTiki Totem\nSunflower Land Collectibles\n3 USDC\n$3.00\n--\n5\nnickname808\n---\n2d ago\nApril 3, 2023 at 3:54 PM',
 'Offer\nCanceled\nLunar Calendar\nSunflower Land Collectibles\n3.600 USDC\n$3.60\n--\n8\nnickname808\n---\n1d ago\nApril 4, 2023 at 10:42 AM',
 'Offer\nLunar Calendar\nSunflower Land Collectibles\n3.720 USDC\n$3.72\n--\n2\nnickname808\n---\n1d ago\nApril 4, 2023 at 3:03 PM',
 'Offer\nLunar Calendar\nSunflower Land Collec

In [10]:
data_all = []

In [11]:
# converting strings to list 
for d_s in data_str:
    data_all.append(d_s.split("\n"))

In [12]:
# removing unwanted values
for d_a in data_all:
    if 'sell' in d_a:
        d_a.remove('sell')

In [13]:
status = ['Accepted', 'Canceled', 'Expired', 'Sold']

In [14]:
# filling values that are not available as information
for d_a in data_all:
    if d_a[1] not in status:
        d_a.insert(1, 'Not Available')
        
for d_a in data_all:
    if d_a[6] == "--":
        d_a[6] = "Not Available"
    if d_a[9] == "---":
        d_a[9] = "Not Available"

In [15]:
data_all[-2:]

[['List',
  'Not Available',
  'Billionaire Skull Club (ETH) #7684',
  'BillionaireSkullClub',
  '0.087 ETH',
  '$166.59',
  'Not Available',
  '1',
  'nickname808',
  'Not Available',
  '1y ago',
  'April 15, 2022 at 10:10 PM'],
 ['List',
  'Not Available',
  'PunksUnchained #7730',
  'Punks Unchained (ETH)',
  '0.015 ETH',
  '$28.72',
  'Not Available',
  '1',
  'nickname808',
  'Not Available',
  '1y ago',
  'April 15, 2022 at 12:09 PM']]

In [16]:
df = pd.DataFrame(data_all, columns=["Event Type", "Status", "NFT", "Collection", "Price", "Price (in dollar)", "Rarity", "Quantity", "From", "To", "Time", "Timestamp", "IDK"])

In [17]:
df.drop(df.columns[-1], inplace=True, axis=1)

In [18]:
df.shape

(185, 12)

In [19]:
df.head()

,Event Type,Status,NFT,Collection,Price,Price (in dollar),Rarity,Quantity,From,To,Time,Timestamp
0,Offer,Not Available,Lunar Calendar,Sunflower Land Collectibles,3.900 USDC,$3.90,Not Available,4,nickname808,Not Available,1d ago,"April 4, 2023 at 4:52 PM"
1,Offer,Not Available,Lunar Calendar,Sunflower Land Collectibles,3.700 USDC,$3.70,Not Available,2,nickname808,Not Available,1d ago,"April 4, 2023 at 3:02 PM"
2,Offer,Canceled,Lunar Calendar,Sunflower Land Collectibles,3.820 USDC,$3.82,Not Available,5,nickname808,Not Available,2d ago,"April 3, 2023 at 9:10 PM"
3,Offer,Canceled,Lunar Calendar,Sunflower Land Collectibles,3.800 USDC,$3.80,Not Available,5,nickname808,Not Available,2d ago,"April 3, 2023 at 3:55 PM"
4,Offer,Not Available,Tiki Totem,Sunflower Land Collectibles,3 USDC,$3.00,Not Available,5,nickname808,Not Available,2d ago,"April 3, 2023 at 3:54 PM"


In [20]:
df.isnull().sum()

Event Type           0
Status               0
NFT                  0
Collection           0
Price                0
Price (in dollar)    0
Rarity               0
Quantity             0
From                 0
To                   0
Time                 0
Timestamp            0
dtype: int64

In [21]:
df.head()

,Event Type,Status,NFT,Collection,Price,Price (in dollar),Rarity,Quantity,From,To,Time,Timestamp
0,Offer,Not Available,Lunar Calendar,Sunflower Land Collectibles,3.900 USDC,$3.90,Not Available,4,nickname808,Not Available,1d ago,"April 4, 2023 at 4:52 PM"
1,Offer,Not Available,Lunar Calendar,Sunflower Land Collectibles,3.700 USDC,$3.70,Not Available,2,nickname808,Not Available,1d ago,"April 4, 2023 at 3:02 PM"
2,Offer,Canceled,Lunar Calendar,Sunflower Land Collectibles,3.820 USDC,$3.82,Not Available,5,nickname808,Not Available,2d ago,"April 3, 2023 at 9:10 PM"
3,Offer,Canceled,Lunar Calendar,Sunflower Land Collectibles,3.800 USDC,$3.80,Not Available,5,nickname808,Not Available,2d ago,"April 3, 2023 at 3:55 PM"
4,Offer,Not Available,Tiki Totem,Sunflower Land Collectibles,3 USDC,$3.00,Not Available,5,nickname808,Not Available,2d ago,"April 3, 2023 at 3:54 PM"


In [22]:
# sorting the dataframe according to time of activity
df["Timestamp"] = pd.to_datetime(df["Timestamp"])
df.sort_values("Timestamp", inplace=True, ascending=False)

In [23]:
df.head()

,Event Type,Status,NFT,Collection,Price,Price (in dollar),Rarity,Quantity,From,To,Time,Timestamp
9,Offer,Not Available,Lunar Calendar,Sunflower Land Collectibles,4.050 USDC,$4.05,Not Available,5,nickname808,Not Available,2h ago,2023-04-05 15:29:00
8,Offer,Not Available,Lunar Calendar,Sunflower Land Collectibles,3.870 USDC,$3.87,Not Available,2,nickname808,Not Available,1d ago,2023-04-04 16:53:00
0,Offer,Not Available,Lunar Calendar,Sunflower Land Collectibles,3.900 USDC,$3.90,Not Available,4,nickname808,Not Available,1d ago,2023-04-04 16:52:00
15,Offer,Not Available,Lunar Calendar,Sunflower Land Collectibles,3.880 USDC,$3.88,Not Available,2,nickname808,Not Available,1d ago,2023-04-04 16:52:00
6,Offer,Not Available,Lunar Calendar,Sunflower Land Collectibles,3.720 USDC,$3.72,Not Available,2,nickname808,Not Available,1d ago,2023-04-04 15:03:00


In [24]:
df["Timestamp"] = df["Timestamp"].dt.strftime("%B %d, %Y at %I:%M %p")

In [25]:
df.head()

,Event Type,Status,NFT,Collection,Price,Price (in dollar),Rarity,Quantity,From,To,Time,Timestamp
9,Offer,Not Available,Lunar Calendar,Sunflower Land Collectibles,4.050 USDC,$4.05,Not Available,5,nickname808,Not Available,2h ago,"April 05, 2023 at 03:29 PM"
8,Offer,Not Available,Lunar Calendar,Sunflower Land Collectibles,3.870 USDC,$3.87,Not Available,2,nickname808,Not Available,1d ago,"April 04, 2023 at 04:53 PM"
0,Offer,Not Available,Lunar Calendar,Sunflower Land Collectibles,3.900 USDC,$3.90,Not Available,4,nickname808,Not Available,1d ago,"April 04, 2023 at 04:52 PM"
15,Offer,Not Available,Lunar Calendar,Sunflower Land Collectibles,3.880 USDC,$3.88,Not Available,2,nickname808,Not Available,1d ago,"April 04, 2023 at 04:52 PM"
6,Offer,Not Available,Lunar Calendar,Sunflower Land Collectibles,3.720 USDC,$3.72,Not Available,2,nickname808,Not Available,1d ago,"April 04, 2023 at 03:03 PM"


In [28]:
df.to_csv("__nickname808.csv")
df.to_excel("__nickname808.xls")

<ipython-input-28-940c160a2c80>:2: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df.to_excel("__nickname808.xls")
